**Import needed packages/modules**

In [ ]:
# Cell 1
import matplotlib.pyplot as plt
import numpy as np
from numba import int64, float64, vectorize

**Declare a numba accelerated function that computes the Halton QRNG**
1. The parameter $n$ is an integer of any size
2. The parameter $p$ is a prime number

In [ ]:
# Cell 2
@vectorize([float64(int64, int64)], nopython=True)
def halton(n, p):
    h, f = 0, 1
    while n > 0:
        f = f / p
        h += (n % p) * f
        n = int(n / p)
    return h

**Set the number of random $dots$ (samples) to take**

In [ ]:
# Cell 3
total_dots = 30_000

**Take $n$ "random" samples of 2D Cartesian points $(x,y)$ using the Halton sequence**
1. Scale the results so $-1\le x_{rng}\le 1$ and $0\le y_{rng}\leq 0.5$
2. The sample area is thus $(-1...1)\times(0...\frac{1}{2})=1$


In [ ]:
# Cell 4
x = (1 - halton(np.arange(total_dots), 2)) * 2.0 - 1.0
y = (1 - halton(np.arange(total_dots), 3)) * 0.5
print(x)
print(y)

**Create an array $d$ containing $y_{rnd}-f(x_{rnd})$**\
Here $f(x)\equiv$ the Gaussian Standard Normal PDF


In [ ]:
# Cell 5

def f(x):
    # Standard Normal PDF
    return 1.0 / np.sqrt(2.0 * np.pi) * np.exp(-np.power(x, 2) / 2.0)

d = y - f(x)
print(d)

**Create arrays of $(x,y)$ coordinates that are "above" or "on or below" the curve**\
if $d>0$ then the sample point is "above" the curve

In [ ]:
# Cell 6
x_in = x[d <= 0.0]
y_in = y[d <= 0.0]
x_out = x[d > 0.0]
y_out = y[d > 0.0]

**Calculate the absolute percent error in the area estimation**
1. The actual/expected definite _non-analytic_ integral is $0.682689492...$
2. The observed/estimated area using the Monte Carlo formulation $\large=1\times\frac{dots_{\ inside}}{dots_{\ total}}$


In [ ]:
# Cell 7
act = 0.682689492
est = 1 * np.count_nonzero(d <= 0.0) / total_dots
err = np.abs((est - act) / act)

print(f"dots = {total_dots:,}")
print(f"act = {act:.6f}")
print(f"est = {est:.6f}")
print(f"err = {err:.5%}")

**Display the scatter plot of the Monte Carlo estimation**\
Include a line graph of the Std Normal PDF to highlight the integrand

In [ ]:
# Cell 8

act_x = np.linspace(-4, 4, 100)
act_y = f(act_x)

plt.figure(figsize=(10, 8))
plt.scatter(x_in, y_in, color="red", marker=".", s=0.5)
plt.scatter(x_out, y_out, color="blue", marker=".", s=0.5)
plt.plot(
    act_x, act_y, color="green",
    label=r"$\frac{1}{\sqrt{2\pi}}e^{\frac{-x^2}{2}}$"
)
plt.title("Standard Normal CDF")
plt.axhline(0, color="gray")
plt.axvline(0, color="gray")
plt.xlim(-4.0, 4.0)
plt.ylim(-0.1, 0.6)
plt.xlabel("x")
plt.ylabel("PDF")
plt.legend(loc="upper right", fontsize="12")
plt.tight_layout()
plt.show()